In [1]:
# Programmer: Allen Chang

# This program will map precinct number and precinct population to nyc high schools by DBN (code used by the NYC Department of Education to uniquely identify schools)

# It will call maps.nyc.gov api to get the precinct number for each high school

# Precinct population data (2010 Census) was taken from: http://johnkeefe.net/nyc-police-precinct-and-census-data
# Direct link to data: https://fusiontables.google.com/DataSource?dsrcid=767562#rows:id=1

# dependencies
import requests
import pandas as pd
import numpy as np

# files to load
hs_directory = "../Resources/Precinct_Needed.xlsx" # dbn, address and borough of high schools in the performance data
precinct_pop_summary = "../Resources/NYC_Blocks_2010CensusData_Plus_Precincts-summary.csv"

# read files and store into pandas data frames
high_schools = pd.read_excel(hs_directory)
precinct_pop = pd.read_csv(precinct_pop_summary)

In [2]:
# select required columns
precinct_pop = precinct_pop[["precinct", "SUM(P0010001)"]]

In [3]:
# clean data
high_schools["Address"] = high_schools["Address"].replace('- ', '-', regex=True)
high_schools["Borough Name"] = high_schools["Borough Name"].replace('STATEN IS', 'STATEN ISLAND', regex=True)

# rename column headers
precinct_pop = precinct_pop.rename(index=str, columns={"precinct": "Precinct",
                                                       "SUM(P0010001)": "Population"})

# drop rows with missing data
precinct_pop = precinct_pop.dropna(how='any')

In [4]:
# replace spaces with %20 prior to creating query url
high_schools["Address"] = high_schools["Address"].replace(' ', '%20', regex=True)

# add column to store precinct numbers
high_schools["Precinct"] = np.nan

In [5]:
# call api for each hs address and store precinct number
i = 0
for row in high_schools["Address"]:
    address = high_schools["Address"][i]
    borough = high_schools["Borough Name"][i].strip()
    url = f"https://maps.nyc.gov/geoclient/v1/search.json?app_key=E2857975AA57366BC&app_id=nyc-gov-nypd&input={address},%20{borough}"
    query = requests.get(url).json()
    high_schools["Precinct"][i] = query["results"][0]["response"]["policePrecinct"]
    i += 1

C:\Users\Allen\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [6]:
# remove unneeded columns
high_schools = high_schools.drop(columns=["Address", "Borough Name"])

In [7]:
# merge data
precinct_key = pd.merge(high_schools, precinct_pop, on="Precinct")

In [9]:
# export to excel file
precinct_key.to_excel("../Resources/Precinct_Key.xlsx", index=False)